In [1]:
# Install TensorFlow with GPU support and plotting libraries
%pip install "tensorflow[and-cuda]" numpy matplotlib scipy scikit-learn

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached grpcio-1.76.0-cp310-cp310-win_amd64.whl.metadata (3.8 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached tensorflow-2.17.0-cp310-cp310-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
INFO: pip is st

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
exceptiongroup 1.3.1 requires typing-extensions>=4.6.0; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
ipython 8.38.0 requires typing_extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.12.5 requires typing-extensions>=4.14.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.41.5 requires typing-extensions>=4.14.1, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.10.0 requires typing-extensions>=4.10.0, but you have typing-extensions 4.5.0 which is incompatible.
typing-inspection 0.4.2 requires typing-extensions>=4.12.0, but you have typing-extensions 4.5.0 which is incompatible.
wandb 0.24.1 requires typing-extensions<5,>=4.8, but you have typing-extensions 4.5.0 which i

In [2]:
import sys

# 1. Print where the notebook is actually running from
print(f"Running from: {sys.executable}")

# 2. Force install into THIS specific python
!{sys.executable} -m pip install "tensorflow[and-cuda]" numpy matplotlib scipy scikit-learn

Running from: c:\Users\ishaa\OneDrive\Desktop\CrossVission-Attacks\venv\Scripts\python.exe



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Imports & Configuration

In [6]:
import os
import tarfile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Tuple, Dict, Callable

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG19, ResNet50, DenseNet121
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Apply a global dark plotting theme so all figures and saved images have a black background
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'black'
plt.rcParams['savefig.facecolor'] = 'black'
plt.rcParams['text.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['legend.facecolor'] = 'black'
plt.rcParams['legend.edgecolor'] = 'white'

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.version.VERSION}")

TensorFlow version: 2.13.1
Keras version: 2.13.1


## 2. GPU Configuration & Mixed Precision

In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Found {len(gpus)} GPU(s): {[gpu.name for gpu in gpus]}")
        print("Memory growth enabled for all GPUs")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")
else:
    print("WARNING: No GPU found. Training will be slow on CPU.")

tf.keras.mixed_precision.set_global_policy('mixed_float16')
print(f"Mixed precision policy: {tf.keras.mixed_precision.global_policy().name}")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Mixed precision policy: mixed_float16


## 3. Global Configuration

In [10]:
CONFIG = {
    'zip_file': '../caltech-101.zip',  # Path relative to Model Training folder
    'extract_dir': 'caltech101_data',
    'img_size': (224, 224),
    'batch_size': 32,
    'epochs': 50,
    'train_split': 0.8,
    'val_split': 0.1,
    'test_split': 0.1,
    'seed': 42,
    'learning_rate': 1e-4,
    'patience_early_stop': 10,
    'patience_lr_reduce': 5,
}

MODELS_TO_TRAIN = ['VGG19', 'ResNet50', 'DenseNet121']
CHECKPOINT_DIR = Path('checkpoints')
CHECKPOINT_DIR.mkdir(exist_ok=True)

np.random.seed(CONFIG['seed'])
tf.random.set_seed(CONFIG['seed'])

In [ ]:
def extract_zip_if_needed(zip_path: str, extract_dir: str) -> Path:
    """Extract zip file if not already extracted."""
    extract_path = Path(extract_dir)
    
    if extract_path.exists() and any(extract_path.iterdir()):
        print(f"Data already extracted at: {extract_path}")
        return extract_path
    
    if not Path(zip_path).exists():
        raise FileNotFoundError(f"Zip file not found: {zip_path}")
    
    print(f"Extracting {zip_path}...")
    extract_path.mkdir(parents=True, exist_ok=True)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path=extract_path)
    
    print(f"Extraction complete: {extract_path}")
    return extract_path

data_root = extract_zip_if_needed(CONFIG['zip_file'], CONFIG['extract_dir'])

def find_image_root(base_path: Path) -> Path:
    """Find the actual root containing class folders with images."""
    # First, look for common Caltech-101 directory structures
    possible_roots = [
        base_path / '101_ObjectCategories',
        base_path / 'caltech-101' / '101_ObjectCategories',
        base_path / 'caltech101' / '101_ObjectCategories',
    ]
    
    for root in possible_roots:
        if root.exists() and root.is_dir():
            print(f"Found image root at: {root}")
            return root
    
    # Recursive search for 101_ObjectCategories anywhere in the tree
    for obj_cat in base_path.rglob('101_ObjectCategories'):
        if obj_cat.is_dir():
            print(f"Found 101_ObjectCategories at: {obj_cat}")
            return obj_cat
    
    # Fallback: search for image files and go up to find class root
    for item in base_path.rglob('*.jpg'):
        if item.is_file():
            # Go up to the class folder's parent (the root with all classes)
            class_folder = item.parent
            root = class_folder.parent
            if root.is_dir() and len(list(root.iterdir())) > 10:  # Should have many class folders
                print(f"Found image root via image search: {root}")
                return root
    
    # Another fallback: find directory with many subdirectories (class folders)
    for subdir in base_path.rglob('*'):
        if subdir.is_dir() and subdir.name != '__MACOSX':
            subdirs = [s for s in subdir.iterdir() if s.is_dir() and s.name != '__MACOSX']
            if len(subdirs) > 50:  # Caltech-101 has 101+ classes
                print(f"Found root with {len(subdirs)} class folders: {subdir}")
                return subdir
    
    print(f"WARNING: Could not find image root, using base path: {base_path}")
    return base_path

IMAGE_ROOT = find_image_root(data_root)
print(f"Image root directory: {IMAGE_ROOT}")

# List contents to verify
print(f"\nContents of extract directory:")
for item in sorted(data_root.iterdir())[:10]:
    print(f"  {item.name}")

Data already extracted at: caltech101_data
Image root directory: caltech101_data

Contents of extract directory:
  __MACOSX
  caltech-101


## 5. Dataset Discovery & Statistics

In [ ]:
def get_class_names_and_counts(image_root: Path) -> Tuple[list, dict]:
    """Get class names and image counts per class."""
    # Filter out system/hidden folders
    exclude_dirs = {'__MACOSX', '.DS_Store', 'BACKGROUND_Google', '__pycache__'}
    
    class_names = sorted([
        d.name for d in image_root.iterdir() 
        if d.is_dir() and d.name not in exclude_dirs and not d.name.startswith('.')
    ])
    class_counts = {}
    
    for class_name in class_names:
        class_dir = image_root / class_name
        count = len([f for f in class_dir.iterdir() 
                     if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']])
        class_counts[class_name] = count
    
    return class_names, class_counts

CLASS_NAMES, CLASS_COUNTS = get_class_names_and_counts(IMAGE_ROOT)
NUM_CLASSES = len(CLASS_NAMES)
TOTAL_IMAGES = sum(CLASS_COUNTS.values())

print(f"Number of classes: {NUM_CLASSES}")
print(f"Total images: {TOTAL_IMAGES}")
print(f"\nSample classes: {CLASS_NAMES[:10]}...")
print(f"\nClass distribution (first 10):")
for cls in list(CLASS_COUNTS.keys())[:10]:
    print(f"  {cls}: {CLASS_COUNTS[cls]} images")

Number of classes: 2
Total images: 0

Sample classes: ['__MACOSX', 'caltech-101']...

Class distribution (first 10):
  __MACOSX: 0 images
  caltech-101: 0 images


## 6. Create tf.data.Dataset with Train/Val/Test Splits

In [ ]:
def collect_all_image_paths_and_labels(image_root: Path, class_names: list) -> Tuple[np.ndarray, np.ndarray]:
    """Collect all image paths and their corresponding labels."""
    all_paths = []
    all_labels = []
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}
    
    for class_name in class_names:
        class_dir = image_root / class_name
        for img_path in class_dir.iterdir():
            if img_path.is_file() and img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']:
                all_paths.append(str(img_path))
                all_labels.append(class_to_idx[class_name])
    
    return np.array(all_paths), np.array(all_labels)

all_paths, all_labels = collect_all_image_paths_and_labels(IMAGE_ROOT, CLASS_NAMES)

indices = np.arange(len(all_paths))
np.random.shuffle(indices)
all_paths = all_paths[indices]
all_labels = all_labels[indices]

train_size = int(CONFIG['train_split'] * len(all_paths))
val_size = int(CONFIG['val_split'] * len(all_paths))

train_paths, train_labels = all_paths[:train_size], all_labels[:train_size]
val_paths, val_labels = all_paths[train_size:train_size+val_size], all_labels[train_size:train_size+val_size]
test_paths, test_labels = all_paths[train_size+val_size:], all_labels[train_size+val_size:]

print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Test samples: {len(test_paths)}")

Train samples: 7315
Validation samples: 914
Test samples: 915


## 7. Dataset Pipeline Factory (with Model-Specific Preprocessing)

In [ ]:
# Ensure compute_class_weight is available (install scikit-learn if needed)
try:
    from sklearn.utils.class_weight import compute_class_weight
except Exception:
    import sys, subprocess
    print("scikit-learn not found — installing scikit-learn...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    from sklearn.utils.class_weight import compute_class_weight

PREPROCESS_FUNCTIONS = {
    'VGG19': vgg_preprocess,
    'ResNet50': resnet_preprocess,
    'DenseNet121': densenet_preprocess,
}

# Compute class weights to handle imbalanced classes
CLASS_WEIGHTS = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
CLASS_WEIGHT_DICT = dict(enumerate(CLASS_WEIGHTS))
print(f"Class weights computed | Range: [{CLASS_WEIGHTS.min():.3f}, {CLASS_WEIGHTS.max():.3f}]")

# Cache for datasets to avoid recreating them multiple times
_dataset_cache = {}

def create_dataset_pipeline(
    paths: np.ndarray,
    labels: np.ndarray,
    preprocess_fn: Callable,
    img_size: Tuple[int, int],
    batch_size: int,
    shuffle: bool = False,
    augment: bool = False,
    cache: bool = False
) -> tf.data.Dataset:
    """Create an optimized tf.data pipeline with model-specific preprocessing."""
    
    def load_and_preprocess(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32)
        img = preprocess_fn(img)
        return img, label
    
    def augment_image(img, label):
        """Enhanced augmentation: flips, brightness, contrast, saturation, random crop."""
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.2)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        # Random crop (simulates zoom) then resize back
        crop_frac = tf.random.uniform([], 0.85, 1.0)
        h, w = img_size
        new_h = tf.cast(tf.cast(h, tf.float32) * crop_frac, tf.int32)
        new_w = tf.cast(tf.cast(w, tf.float32) * crop_frac, tf.int32)
        img = tf.image.random_crop(img, [new_h, new_w, 3])
        img = tf.image.resize(img, img_size)
        return img, label
    
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=min(len(paths), 10000), seed=CONFIG['seed'], reshuffle_each_iteration=True)
    
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if cache:
        dataset = dataset.cache()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

def get_datasets_for_model(model_name: str, use_cache: bool = True) -> Tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    """Create train, val, test datasets with model-specific preprocessing (cached)."""
    cache_key = model_name
    
    if use_cache and cache_key in _dataset_cache:
        return _dataset_cache[cache_key]
    
    preprocess_fn = PREPROCESS_FUNCTIONS[model_name]
    
    train_ds = create_dataset_pipeline(
        train_paths, train_labels, preprocess_fn,
        CONFIG['img_size'], CONFIG['batch_size'],
        shuffle=True, augment=True, cache=False  # Don't cache augmented data
    )
    val_ds = create_dataset_pipeline(
        val_paths, val_labels, preprocess_fn,
        CONFIG['img_size'], CONFIG['batch_size'],
        shuffle=False, augment=False, cache=True  # Cache validation data
    )
    test_ds = create_dataset_pipeline(
        test_paths, test_labels, preprocess_fn,
        CONFIG['img_size'], CONFIG['batch_size'],
        shuffle=False, augment=False, cache=True  # Cache test data
    )
    
    if use_cache:
        _dataset_cache[cache_key] = (train_ds, val_ds, test_ds)
    
    return train_ds, val_ds, test_ds

def clear_dataset_cache():
    """Clear the dataset cache to free memory."""
    global _dataset_cache
    _dataset_cache = {}
    print("Dataset cache cleared.")

print("Dataset pipeline factory created with enhanced augmentation and caching.")

Class weights computed | Range: [0.112, 3.118]
Dataset pipeline factory created with enhanced augmentation and caching.


## 8. Model Factory

In [ ]:
BASE_MODELS = {
    'VGG19': VGG19,
    'ResNet50': ResNet50,
    'DenseNet121': DenseNet121,
}

# Number of layers to unfreeze for fine-tuning
FINE_TUNE_LAYERS = 20

def create_model(model_name: str, num_classes: int, img_size: Tuple[int, int], fine_tune: bool = True) -> Model:
    """Create a transfer learning model with fine-tuning of last 20 layers.
    
    Args:
        model_name: Name of the base model ('VGG19', 'ResNet50', 'DenseNet121')
        num_classes: Number of output classes
        img_size: Input image size (height, width)
        fine_tune: If True, unfreeze last FINE_TUNE_LAYERS layers for training
    
    Returns:
        Compiled Keras model with fine-tuning enabled
    """
    
    base_model_class = BASE_MODELS[model_name]
    
    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3),
        pooling=None
    )
    
    # Freeze all layers first
    for layer in base_model.layers:
        layer.trainable = False
    
    # Unfreeze last FINE_TUNE_LAYERS layers for genuine training on Caltech-101
    if fine_tune:
        for layer in base_model.layers[-FINE_TUNE_LAYERS:]:
            layer.trainable = True
        print(f"  Fine-tuning enabled: Last {FINE_TUNE_LAYERS} layers unfrozen")
    
    inputs = keras.Input(shape=(*img_size, 3))
    x = base_model(inputs, training=fine_tune)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs, outputs, name=f"{model_name}_transfer")
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

print("Model factory created with fine-tuning support.")
print(f"Models available: {list(BASE_MODELS.keys())}")
print(f"Fine-tune layers: Last {FINE_TUNE_LAYERS} layers will be unfrozen for training")

Model factory created.
Models available: ['VGG19', 'ResNet101', 'DenseNet121']


## 9. Callbacks Factory

In [ ]:
# Minimum epochs before early stopping can trigger
MIN_EPOCHS_BEFORE_EARLY_STOP = 20

def get_callbacks(model_name: str) -> list:
    """Create callbacks for training with minimum epoch threshold.
    
    Early stopping won't trigger until at least MIN_EPOCHS_BEFORE_EARLY_STOP
    epochs have completed, ensuring sufficient training on Caltech-101.
    """
    
    checkpoint_path = CHECKPOINT_DIR / f"{model_name}_best.keras"
    
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=CONFIG['patience_early_stop'],
            restore_best_weights=True,
            verbose=1,
            start_from_epoch=MIN_EPOCHS_BEFORE_EARLY_STOP  # Don't stop before epoch 20
        ),
        ModelCheckpoint(
            filepath=str(checkpoint_path),
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=CONFIG['patience_lr_reduce'],
            min_lr=1e-7,
            verbose=1
        ),
    ]
    
    return callbacks

print("Callbacks factory created.")
print(f"Minimum epochs before early stopping: {MIN_EPOCHS_BEFORE_EARLY_STOP}")

Callbacks factory created.


## 10. Training Loop

In [ ]:
# Initialize dictionaries to store results
training_histories: Dict[str, keras.callbacks.History] = {}
trained_models: Dict[str, Model] = {}

print("Training storage initialized.")

Training storage initialized.


## 10.1 Train VGG19

In [ ]:
# Train VGG19
model_name = 'VGG19'

print("\n" + "="*80)
print(f"Training {model_name}")
print("="*80)

tf.keras.backend.clear_session()

train_ds, val_ds, test_ds = get_datasets_for_model(model_name)

# Check if checkpoint exists and load it, otherwise create new model
checkpoint_path = CHECKPOINT_DIR / f"{model_name}_best.keras"
if checkpoint_path.exists():
    print(f"\nLoading existing model from: {checkpoint_path}")
    model = keras.models.load_model(checkpoint_path)
    print("Model loaded successfully!")
else:
    print(f"\nNo checkpoint found. Creating new {model_name} model...")
    model = create_model(model_name, NUM_CLASSES, CONFIG['img_size'])

print(f"\n{model_name} Summary:")
print(f"  Total params: {model.count_params():,}")
trainable_params = sum([tf.reduce_prod(v.shape).numpy() for v in model.trainable_variables])
print(f"  Trainable params: {trainable_params:,}")
print(f"  Fine-tuned layers: Last {FINE_TUNE_LAYERS} layers unfrozen")
print(f"  Minimum epochs before early stopping: {MIN_EPOCHS_BEFORE_EARLY_STOP}")

callbacks = get_callbacks(model_name)

print(f"\n--- Training with fine-tuning (last {FINE_TUNE_LAYERS} layers) ---")
print(f"Early Stopping: patience={CONFIG['patience_early_stop']}, monitor='val_loss'")
print(f"Early Stopping: start_from_epoch={MIN_EPOCHS_BEFORE_EARLY_STOP} (minimum training guaranteed)")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG['epochs'],
    callbacks=callbacks,
    class_weight=CLASS_WEIGHT_DICT,
    verbose=1
)

training_histories[model_name] = history
trained_models[model_name] = model

print(f"\n{model_name} training complete!")
print(f"Total epochs: {len(history.history['loss'])}")


Training VGG19


I0000 00:00:1770072071.045398  118378 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3398 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9



No checkpoint found. Creating new VGG19 model...

VGG19 Summary:
  Total params: 20,446,630
  Trainable params: 421,222

--- Phase 1: Training classification head ---
Early Stopping: patience=10, monitor='val_loss'
Epoch 1/50


2026-02-03 04:11:13.318048: I external/local_xla/xla/service/service.cc:163] XLA service 0x716290010d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-03 04:11:13.318073: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2026-02-03 04:11:13.551643: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-03 04:11:13.876280: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91801
2026-02-03 04:11:13.954477: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-02-03 04:11:13.

228/229 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.0175 - loss: 5.0671

...


Epoch 1: val_accuracy improved from None to 0.16521, saving model to checkpoints/VGG19_best.keras

Epoch 1: finished saving model to checkpoints/VGG19_best.keras
229/229 ━━━━━━━━━━━━━━━━━━━━ 82s 283ms/step - accuracy: 0.0271 - loss: 4.8654 - val_accuracy: 0.1652 - val_loss: 3.9945 - learning_rate: 1.0000e-04
Epoch 2/50
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.0751 - loss: 4.2922
Epoch 2: val_accuracy improved from 0.16521 to 0.44748, saving model to checkpoints/VGG19_best.keras

Epoch 2: finished saving model to checkpoints/VGG19_best.keras
229/229 ━━━━━━━━━━━━━━━━━━━━ 43s 188ms/step - accuracy: 0.1012 - loss: 4.2041 - val_accuracy: 0.4475 - val_loss: 3.2475 - learning_rate: 1.0000e-04
Epoch 3/50
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.1892 - loss: 3.7958
Epoch 3: val_accuracy improved from 0.44748 to 0.59081, saving model to checkpoints/VGG19_best.keras

Epoch 3: finished saving model to checkpoints/VGG19_best.keras
229/229 ━━━━━━━━━━━━━━━━━━━━ 41s 17

2026-02-03 04:45:39.504251: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-02-03 04:45:39.504281: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-02-03 04:45:39.504289: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-02-03 04:45:40.322591: I external/l

228/229 ━━━━━━━━━━━━━━━━━━━━ 0s 569ms/step - accuracy: 0.8447 - loss: 0.4653

2026-02-03 04:48:29.113003: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.63GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 681ms/step - accuracy: 0.8448 - loss: 0.4652
Epoch 1: val_accuracy did not improve from 0.89716
229/229 ━━━━━━━━━━━━━━━━━━━━ 201s 706ms/step - accuracy: 0.8476 - loss: 0.4549 - val_accuracy: 0.8939 - val_loss: 0.4132 - learning_rate: 1.0000e-05
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - accuracy: 0.8542 - loss: 0.4510
Epoch 2: val_accuracy did not improve from 0.89716
229/229 ━━━━━━━━━━━━━━━━━━━━ 130s 568ms/step - accuracy: 0.8591 - loss: 0.4387 - val_accuracy: 0.8961 - val_loss: 0.4034 - learning_rate: 1.0000e-05
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 554ms/step - accuracy: 0.8497 - loss: 0.4305
Epoch 3: val_accuracy did not improve from 0.89716
229/229 ━━━━━━━━━━━━━━━━━━━━ 131s 573ms/step - accuracy: 0.8559 - loss: 0.4280 - val_accuracy: 0.8972 - val_loss: 0.4026 - learning_rate: 1.0000e-05
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - accuracy: 0.8610 - loss: 0.3934
Epoch 4: val_accuracy did not improve from 0.89716
229/229 

## 10.2 Train ResNet50

In [ ]:
# Train ResNet50
model_name = 'ResNet50'

print("\n" + "="*80)
print(f"Training {model_name}")
print("="*80)

tf.keras.backend.clear_session()

train_ds, val_ds, test_ds = get_datasets_for_model(model_name)

# Check if checkpoint exists and load it, otherwise create new model
checkpoint_path = CHECKPOINT_DIR / f"{model_name}_best.keras"
if checkpoint_path.exists():
    print(f"\nLoading existing model from: {checkpoint_path}")
    model = keras.models.load_model(checkpoint_path)
    print("Model loaded successfully! Resuming training...")
else:
    print(f"\nNo checkpoint found. Creating new {model_name} model...")
    model = create_model(model_name, NUM_CLASSES, CONFIG['img_size'])

print(f"\n{model_name} Summary:")
print(f"  Total params: {model.count_params():,}")
trainable_params = sum([tf.reduce_prod(v.shape).numpy() for v in model.trainable_variables])
print(f"  Trainable params: {trainable_params:,}")
print(f"  Fine-tuned layers: Last {FINE_TUNE_LAYERS} layers unfrozen")
print(f"  Minimum epochs before early stopping: {MIN_EPOCHS_BEFORE_EARLY_STOP}")

callbacks = get_callbacks(model_name)

print(f"\nEarly Stopping: patience={CONFIG['patience_early_stop']}, monitor='val_loss', restore_best_weights=True")
print(f"Early Stopping: start_from_epoch={MIN_EPOCHS_BEFORE_EARLY_STOP} (minimum training guaranteed)")
print(f"LR Reduction: patience={CONFIG['patience_lr_reduce']}, factor=0.5, min_lr=1e-7")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG['epochs'],
    callbacks=callbacks,
    class_weight=CLASS_WEIGHT_DICT,
    verbose=1
)

training_histories[model_name] = history
trained_models[model_name] = model

print(f"Stopped at epoch: {len(history.history['loss'])}/{CONFIG['epochs']}")
print(f"\n{model_name} training complete!")


Training ResNet101

No checkpoint found. Creating new ResNet101 model...

ResNet101 Summary:
  Total params: 43,872,998
  Trainable params: 1,210,726

Early Stopping: patience=10, monitor='val_loss', restore_best_weights=True
LR Reduction: patience=5, factor=0.5, min_lr=1e-7
Epoch 1/50


2026-02-03 05:11:27.489362: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-02-03 05:11:27.918270: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_10319', 56 bytes spill stores, 56 bytes spill loads

2026-02-03 05:11:28.307889: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_10757', 520 bytes spill stores, 520 bytes spill loads



228/229 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.0297 - loss: 5.0476

2026-02-03 05:12:04.978442: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_10742', 4 bytes spill stores, 4 bytes spill loads

2026-02-03 05:12:05.126289: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_10319', 52 bytes spill stores, 52 bytes spill loads



229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.0298 - loss: 5.0459

2026-02-03 05:12:17.183926: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3317', 52 bytes spill stores, 52 bytes spill loads




Epoch 1: val_accuracy improved from None to 0.47812, saving model to checkpoints/ResNet101_best.keras

Epoch 1: finished saving model to checkpoints/ResNet101_best.keras
229/229 ━━━━━━━━━━━━━━━━━━━━ 61s 206ms/step - accuracy: 0.0637 - loss: 4.6594 - val_accuracy: 0.4781 - val_loss: 3.2524 - learning_rate: 1.0000e-04
Epoch 2/50
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.2128 - loss: 3.5421
Epoch 2: val_accuracy improved from 0.47812 to 0.71772, saving model to checkpoints/ResNet101_best.keras

Epoch 2: finished saving model to checkpoints/ResNet101_best.keras
229/229 ━━━━━━━━━━━━━━━━━━━━ 31s 134ms/step - accuracy: 0.2708 - loss: 3.3707 - val_accuracy: 0.7177 - val_loss: 1.9173 - learning_rate: 1.0000e-04
Epoch 3/50
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.4246 - loss: 2.6231
Epoch 3: val_accuracy improved from 0.71772 to 0.80197, saving model to checkpoints/ResNet101_best.keras

Epoch 3: finished saving model to checkpoints/ResNet101_best.keras
229/229 ━━━

## 10.3 Train DenseNet121

In [ ]:
# Train DenseNet121
model_name = 'DenseNet121'

print("\n" + "="*80)
print(f"Training {model_name}")
print("="*80)

tf.keras.backend.clear_session()

train_ds, val_ds, test_ds = get_datasets_for_model(model_name)

# Check if checkpoint exists and load it, otherwise create new model
checkpoint_path = CHECKPOINT_DIR / f"{model_name}_best.keras"
if checkpoint_path.exists():
    print(f"\nLoading existing model from: {checkpoint_path}")
    model = keras.models.load_model(checkpoint_path)
    print("Model loaded successfully! Resuming training...")
else:
    print(f"\nNo checkpoint found. Creating new {model_name} model...")
    model = create_model(model_name, NUM_CLASSES, CONFIG['img_size'])

print(f"\n{model_name} Summary:")
print(f"  Total params: {model.count_params():,}")
trainable_params = sum([tf.reduce_prod(v.shape).numpy() for v in model.trainable_variables])
print(f"  Trainable params: {trainable_params:,}")
print(f"  Fine-tuned layers: Last {FINE_TUNE_LAYERS} layers unfrozen")
print(f"  Minimum epochs before early stopping: {MIN_EPOCHS_BEFORE_EARLY_STOP}")

callbacks = get_callbacks(model_name)

print(f"\nEarly Stopping: patience={CONFIG['patience_early_stop']}, monitor='val_loss', restore_best_weights=True")
print(f"Early Stopping: start_from_epoch={MIN_EPOCHS_BEFORE_EARLY_STOP} (minimum training guaranteed)")
print(f"LR Reduction: patience={CONFIG['patience_lr_reduce']}, factor=0.5, min_lr=1e-7")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG['epochs'],
    callbacks=callbacks,
    class_weight=CLASS_WEIGHT_DICT,
    verbose=1
)

training_histories[model_name] = history
trained_models[model_name] = model

print(f"Stopped at epoch: {len(history.history['loss'])}/{CONFIG['epochs']}")
print(f"\n{model_name} training complete!")

## 11. Evaluation on Test Set

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("\n" + "="*80)
print("TEST SET EVALUATION")
print("="*80)

test_results = {}
model_predictions = {}  # Cache predictions to avoid recomputing

for model_name in MODELS_TO_TRAIN:
    print(f"\nEvaluating {model_name}...")
    
    _, _, test_ds = get_datasets_for_model(model_name)
    model = trained_models[model_name]
    
    # Get predictions once and reuse
    print(f"  Computing predictions...", end=" ", flush=True)
    y_pred_probs = model.predict(test_ds, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = test_labels.astype(int)
    print("Done!")
    
    # Cache for later use
    model_predictions[model_name] = {
        'probs': y_pred_probs,
        'pred': y_pred,
        'true': y_true
    }
    
    # Calculate metrics from predictions
    accuracy = np.mean(y_pred == y_true)
    loss, _ = model.evaluate(test_ds, verbose=0)
    
    test_results[model_name] = {
        'loss': loss,
        'accuracy': accuracy
    }
    
    print(f"  Test Loss: {loss:.4f}")
    print(f"  Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)
print(f"\n{'Model':<15} {'Test Loss':<12} {'Test Accuracy':<15}")
print("-" * 42)
for model_name, results in test_results.items():
    print(f"{model_name:<15} {results['loss']:<12.4f} {results['accuracy']*100:.2f}%")

best_model = max(test_results.keys(), key=lambda k: test_results[k]['accuracy'])
print(f"\nBest performing model: {best_model} with {test_results[best_model]['accuracy']*100:.2f}% accuracy")

# --- Detailed diagnostics for each model (using cached predictions) ---
print("\n" + "="*80)
print("DETAILED DIAGNOSTICS")
print("="*80)

for model_name in MODELS_TO_TRAIN:
    print(f"\n--- {model_name} ---")
    
    # Use cached predictions
    y_pred = model_predictions[model_name]['pred']
    y_true = model_predictions[model_name]['true']
    
    # Confusion matrix - find most confused pairs
    cm = confusion_matrix(y_true, y_pred)
    cm_no_diag = cm.copy()
    np.fill_diagonal(cm_no_diag, 0)
    
    print("Top 5 most confused class pairs (true → pred):")
    flat_idx = np.argsort(cm_no_diag.ravel())[-5:][::-1]
    for idx in flat_idx:
        i, j = divmod(idx, cm.shape[1])
        if cm_no_diag[i, j] > 0:
            print(f"  {CLASS_NAMES[i]} → {CLASS_NAMES[j]}: {cm_no_diag[i, j]}")
    
    # Worst classes by F1
    report = classification_report(y_true, y_pred, target_names=CLASS_NAMES, output_dict=True, zero_division=0)
    worst = sorted([(c, report[c]['f1-score']) for c in CLASS_NAMES], key=lambda x: x[1])[:5]
    print("Worst 5 classes by F1-score:")
    for cls, f1 in worst:
        print(f"  {cls}: {f1:.3f}")

## 12. Misclassification Analysis & Grad-CAM

Visualize misclassified samples and use Grad-CAM to understand where the model focuses its attention.

In [ ]:
def get_misclassified_samples(model_name: str):
    """Get misclassified test samples for a model (uses cached predictions from evaluation)."""
    # Reuse predictions from evaluation cell if available
    if model_name in model_predictions:
        y_pred_probs = model_predictions[model_name]['probs']
        y_pred = model_predictions[model_name]['pred']
        y_true = model_predictions[model_name]['true']
    else:
        # Fallback: compute predictions
        print(f"  Computing predictions for {model_name}...", end=" ", flush=True)
        model = trained_models[model_name]
        _, _, test_ds = get_datasets_for_model(model_name)
        y_pred_probs = model.predict(test_ds, verbose=0)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = test_labels.astype(int)
        model_predictions[model_name] = {'probs': y_pred_probs, 'pred': y_pred, 'true': y_true}
        print("Done!")
    
    misclassified_idx = np.where(y_pred != y_true)[0]
    return misclassified_idx, y_pred, y_pred_probs, y_true

def plot_misclassified(model_name: str, n_show: int = 8):
    """Display misclassified images for a model."""
    misclassified_idx, y_pred, y_pred_probs, y_true = get_misclassified_samples(model_name)
    
    n_show = min(n_show, len(misclassified_idx))
    if n_show == 0:
        print(f"No misclassified samples for {model_name}!")
        return
    
    cols = 4
    rows = (n_show + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(16, 4 * rows))
    axes = axes.flatten() if n_show > 1 else [axes]
    
    for i, idx in enumerate(misclassified_idx[:n_show]):
        img = plt.imread(test_paths[idx])
        axes[i].imshow(img)
        axes[i].axis('off')
        conf = y_pred_probs[idx, y_pred[idx]] * 100
        axes[i].set_title(f"True: {CLASS_NAMES[y_true[idx]]}\nPred: {CLASS_NAMES[y_pred[idx]]} ({conf:.1f}%)", fontsize=9)
    
    for i in range(n_show, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f"{model_name} - Misclassified Samples ({len(misclassified_idx)} total)", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'{model_name.lower()}_misclassified.png', dpi=150, bbox_inches='tight')
    plt.show()

# Cache for feature models to avoid recreating
_feature_model_cache = {}

def visualize_gradcam(model_name: str, img_idx: int = None):
    """Visualize activation heatmap for a misclassified image (optimized)."""
    print(f"  Generating activation map for {model_name}...", end=" ", flush=True)
    
    misclassified_idx, y_pred, y_pred_probs, y_true = get_misclassified_samples(model_name)
    
    if len(misclassified_idx) == 0:
        print(f"No misclassified samples for {model_name}")
        return
    
    idx = misclassified_idx[0] if img_idx is None else img_idx
    img_path = test_paths[idx]
    
    # Load and preprocess image
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, CONFIG['img_size'])
    img = tf.cast(img, tf.float32)
    img_preprocessed = PREPROCESS_FUNCTIONS[model_name](img)
    img_array = tf.expand_dims(img_preprocessed, 0)
    
    # Get or create feature model (cached)
    if model_name not in _feature_model_cache:
        model = trained_models[model_name]
        base = model.layers[1]
        last_conv_name = [l.name for l in base.layers if 'conv' in l.name][-1]
        _feature_model_cache[model_name] = Model(inputs=base.input, outputs=base.get_layer(last_conv_name).output)
    
    feature_model = _feature_model_cache[model_name]
    
    # Get features and create heatmap from activations
    features = feature_model(img_array)
    heatmap = tf.reduce_mean(features, axis=-1)[0]
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    heatmap = heatmap.numpy()
    
    print("Done!")
    
    # Load original image and create overlay
    orig_img = plt.imread(img_path)
    heatmap_resized = np.array(tf.image.resize(heatmap[..., np.newaxis], orig_img.shape[:2]))[:, :, 0]
    heatmap_resized = np.uint8(255 * heatmap_resized)
    
    cmap = plt.cm.jet(heatmap_resized)[:, :, :3]
    overlay = 0.5 * cmap + 0.5 * (orig_img / 255.0 if orig_img.max() > 1 else orig_img)
    overlay = np.clip(overlay, 0, 1)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(orig_img)
    axes[0].set_title(f"Original\nTrue: {CLASS_NAMES[y_true[idx]]}", fontsize=11)
    axes[0].axis('off')
    
    axes[1].imshow(heatmap, cmap='jet')
    axes[1].set_title("Activation Heatmap", fontsize=11)
    axes[1].axis('off')
    
    axes[2].imshow(overlay)
    axes[2].set_title(f"Overlay\nPred: {CLASS_NAMES[y_pred[idx]]} ({y_pred_probs[idx, y_pred[idx]]*100:.1f}%)", fontsize=11)
    axes[2].axis('off')
    
    plt.suptitle(f"{model_name} Activation Analysis", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'{model_name.lower()}_gradcam.png', dpi=150, bbox_inches='tight')
    plt.show()

# Show misclassified samples and activation map for VGG19
print("="*80)
print("VGG19 MISCLASSIFICATION ANALYSIS")
print("="*80)
plot_misclassified('VGG19', n_show=8)
visualize_gradcam('VGG19')

## 13. Training Curves Visualization

In [ ]:
def plot_training_history(histories: Dict[str, keras.callbacks.History], metric: str = 'accuracy'):
    """Plot training curves for all models."""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history.history[metric]) + 1)
        
        axes[0].plot(epochs, history.history[metric], 
                     color=colors[idx], linestyle='-', linewidth=2,
                     label=f'{model_name} (Train)')
        axes[0].plot(epochs, history.history[f'val_{metric}'], 
                     color=colors[idx], linestyle='--', linewidth=2,
                     label=f'{model_name} (Val)')
    
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].legend(loc='lower right', fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([0, 1])
    
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history.history['loss']) + 1)
        
        axes[1].plot(epochs, history.history['loss'], 
                     color=colors[idx], linestyle='-', linewidth=2,
                     label=f'{model_name} (Train)')
        axes[1].plot(epochs, history.history['val_loss'], 
                     color=colors[idx], linestyle='--', linewidth=2,
                     label=f'{model_name} (Val)')
    
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].legend(loc='upper right', fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Training curves saved to: training_curves.png")

plot_training_history(training_histories)

## 14. Individual Model Training Curves

In [ ]:
def plot_individual_model_curves(histories: Dict[str, keras.callbacks.History]):
    """Plot individual training curves for each model."""
    n_models = len(histories)
    fig, axes = plt.subplots(n_models, 2, figsize=(14, 5 * n_models))
    
    if n_models == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history.history['accuracy']) + 1)
        
        axes[idx, 0].plot(epochs, history.history['accuracy'], 'b-', linewidth=2, label='Train')
        axes[idx, 0].plot(epochs, history.history['val_accuracy'], 'r--', linewidth=2, label='Validation')
        axes[idx, 0].set_title(f'{model_name} - Accuracy', fontsize=12, fontweight='bold')
        axes[idx, 0].set_xlabel('Epoch')
        axes[idx, 0].set_ylabel('Accuracy')
        axes[idx, 0].legend()
        axes[idx, 0].grid(True, alpha=0.3)
        axes[idx, 0].set_ylim([0, 1])
        
        axes[idx, 1].plot(epochs, history.history['loss'], 'b-', linewidth=2, label='Train')
        axes[idx, 1].plot(epochs, history.history['val_loss'], 'r--', linewidth=2, label='Validation')
        axes[idx, 1].set_title(f'{model_name} - Loss', fontsize=12, fontweight='bold')
        axes[idx, 1].set_xlabel('Epoch')
        axes[idx, 1].set_ylabel('Loss')
        axes[idx, 1].legend()
        axes[idx, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('individual_training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Individual curves saved to: individual_training_curves.png")

plot_individual_model_curves(training_histories)

## 15. Model Comparison Bar Chart

In [ ]:
def plot_model_comparison(test_results: dict):
    """Bar chart comparing test accuracy across models."""
    models = list(test_results.keys())
    accuracies = [test_results[m]['accuracy'] * 100 for m in models]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = ['#3498db', '#e74c3c', '#2ecc71']
    bars = ax.bar(models, accuracies, color=colors, edgecolor='black', linewidth=1.2)
    
    for bar, acc in zip(bars, accuracies):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax.set_ylabel('Test Accuracy (%)', fontsize=12)
    ax.set_xlabel('Model', fontsize=12)
    ax.set_title('Model Comparison - Test Accuracy', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 100])
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Model comparison saved to: model_comparison.png")

plot_model_comparison(test_results)

## 16. Save Final Models

In [ ]:
FINAL_MODELS_DIR = Path('final_models')
FINAL_MODELS_DIR.mkdir(exist_ok=True)

for model_name, model in trained_models.items():
    save_path = FINAL_MODELS_DIR / f"{model_name}_final.keras"
    model.save(save_path)
    print(f"Saved {model_name} to: {save_path}")

class_names_path = FINAL_MODELS_DIR / 'class_names.txt'
with open(class_names_path, 'w') as f:
    for name in CLASS_NAMES:
        f.write(f"{name}\n")
print(f"Saved class names to: {class_names_path}")

print("\n" + "="*80)
print("TRAINING PIPELINE COMPLETE")
print("="*80)
print(f"\nCheckpoints directory: {CHECKPOINT_DIR}")
print(f"Final models directory: {FINAL_MODELS_DIR}")
print(f"Visualization files: training_curves.png, individual_training_curves.png, model_comparison.png")

## 16.5 Export Models for Adversarial Attacks

Export trained models in TensorFlow SavedModel format with preprocessing metadata for ART/Foolbox compatibility.

In [ ]:
import json

# Create attack models directory
ATTACK_MODELS_DIR = Path('attack_models')
ATTACK_MODELS_DIR.mkdir(exist_ok=True)

# Preprocessing metadata for each model (required for ART/Foolbox)
PREPROCESSING_INFO = {
    'VGG19': {
        'mode': 'caffe',  # BGR, mean subtraction [103.939, 116.779, 123.68]
        'mean': [103.939, 116.779, 123.68],
        'std': [1.0, 1.0, 1.0],
        'input_range': [0, 255],
        'channel_order': 'BGR'
    },
    'ResNet50': {
        'mode': 'caffe',  # BGR, mean subtraction
        'mean': [103.939, 116.779, 123.68],
        'std': [1.0, 1.0, 1.0],
        'input_range': [0, 255],
        'channel_order': 'BGR'
    },
    'DenseNet121': {
        'mode': 'torch',  # RGB, normalize to [0,1] then subtract mean/std
        'mean': [0.485, 0.456, 0.406],
        'std': [0.229, 0.224, 0.225],
        'input_range': [0, 1],
        'channel_order': 'RGB'
    }
}

print("="*80)
print("EXPORTING MODELS FOR ADVERSARIAL ATTACKS")
print("="*80)

for model_name, model in trained_models.items():
    model_dir = ATTACK_MODELS_DIR / model_name
    model_dir.mkdir(exist_ok=True)
    
    # 1. Save as TensorFlow SavedModel format (best for ART/Foolbox)
    savedmodel_path = model_dir / 'saved_model'
    model.export(str(savedmodel_path))
    print(f"\n{model_name}:")
    print(f"  SavedModel: {savedmodel_path}")
    
    # 2. Also save as .keras for easy loading
    keras_path = model_dir / f'{model_name}_attack.keras'
    model.save(keras_path)
    print(f"  Keras format: {keras_path}")
    
    # 3. Save preprocessing metadata
    metadata = {
        'model_name': model_name,
        'input_shape': [224, 224, 3],
        'num_classes': NUM_CLASSES,
        'class_names_file': '../final_models/class_names.txt',
        'preprocessing': PREPROCESSING_INFO[model_name],
        'training_info': {
            'dataset': 'Caltech-101',
            'total_images': TOTAL_IMAGES,
            'fine_tuned_layers': FINE_TUNE_LAYERS,
            'batch_size': CONFIG['batch_size'],
            'epochs_trained': len(training_histories[model_name].history['loss']),
        }
    }
    
    metadata_path = model_dir / 'preprocessing.json'
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"  Metadata: {metadata_path}")

# Save training configuration for reproducibility
config_path = ATTACK_MODELS_DIR / 'training_config.json'
with open(config_path, 'w') as f:
    json.dump({
        'config': CONFIG,
        'fine_tune_layers': FINE_TUNE_LAYERS,
        'min_epochs_before_early_stop': MIN_EPOCHS_BEFORE_EARLY_STOP,
        'models': list(trained_models.keys()),
        'num_classes': NUM_CLASSES,
    }, f, indent=2)
print(f"\nTraining config: {config_path}")

print("\n" + "="*80)
print("EXPORT COMPLETE - Models ready for adversarial attacks")
print("="*80)
print(f"\nAttack models directory: {ATTACK_MODELS_DIR}")
print("\nTo use with ART (Adversarial Robustness Toolbox):")
print("  from art.estimators.classification import TensorFlowV2Classifier")
print("  import tensorflow as tf")
print("  model = tf.saved_model.load('attack_models/VGG19/saved_model')")

## 17. Individual Model Predictions

Use these cells to make predictions with each model separately. You can provide an image path or use a random test image.

In [ ]:
# Helper function for single image prediction
def predict_single_image(model, model_name: str, image_path: str, top_k: int = 5):
    """
    Make a prediction on a single image using a specific model.
    
    Args:
        model: The trained Keras model
        model_name: Name of the model (for preprocessing selection)
        image_path: Path to the image file
        top_k: Number of top predictions to show
    
    Returns:
        Dictionary with predictions
    """
    # Load and preprocess image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, CONFIG['img_size'])
    img = tf.cast(img, tf.float32)
    
    # Apply model-specific preprocessing
    preprocess_fn = PREPROCESS_FUNCTIONS[model_name]
    img = preprocess_fn(img)
    
    # Add batch dimension
    img = tf.expand_dims(img, 0)
    
    # Predict
    predictions = model.predict(img, verbose=0)[0]
    
    # Get top-k predictions
    top_indices = np.argsort(predictions)[-top_k:][::-1]
    
    results = {
        'image_path': image_path,
        'model': model_name,
        'predictions': []
    }
    
    for idx in top_indices:
        results['predictions'].append({
            'class': CLASS_NAMES[idx],
            'confidence': float(predictions[idx])
        })
    
    return results

def display_prediction(results: dict, show_image: bool = True):
    """Display prediction results with optional image."""
    print(f"\n{'='*60}")
    print(f"Model: {results['model']}")
    print(f"Image: {results['image_path']}")
    print(f"{'='*60}")
    
    if show_image:
        img = plt.imread(results['image_path'])
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predicted: {results['predictions'][0]['class']}")
        plt.show()
    
    print(f"\nTop {len(results['predictions'])} Predictions:")
    print("-" * 40)
    for i, pred in enumerate(results['predictions'], 1):
        confidence = pred['confidence'] * 100
        bar = '█' * int(confidence / 5) + '░' * (20 - int(confidence / 5))
        print(f"{i}. {pred['class']:<25} {bar} {confidence:.2f}%")

print("Prediction helper functions loaded.")

## 16.1 Predict with VGG19

In [ ]:
# ============================================
# PREDICT WITH VGG19
# ============================================

# Option 1: Use a random test image
random_idx = np.random.randint(0, len(test_paths))
sample_image_path = test_paths[random_idx]
true_label = CLASS_NAMES[test_labels[random_idx]]

# Option 2: Specify your own image path (uncomment and modify)
# sample_image_path = "path/to/your/image.jpg"

# Load model (from memory or checkpoint)
if 'VGG19' in trained_models:
    vgg19_model = trained_models['VGG19']
else:
    vgg19_checkpoint = CHECKPOINT_DIR / "VGG19_best.keras"
    vgg19_model = keras.models.load_model(vgg19_checkpoint)
    print(f"Loaded VGG19 from: {vgg19_checkpoint}")

# Make prediction
vgg19_results = predict_single_image(vgg19_model, 'VGG19', sample_image_path, top_k=5)
display_prediction(vgg19_results)

print(f"\nTrue label: {true_label}")

## 16.2 Predict with ResNet50

In [ ]:
# ============================================
# PREDICT WITH ResNet50
# ============================================

# Option 1: Use a random test image
# random_idx = np.random.randint(0, len(test_paths))
# sample_image_path = test_paths[random_idx]
# true_label = CLASS_NAMES[test_labels[random_idx]]

# Option 2: Specify your own image path (uncomment and modify)
sample_image_path = "caltech101_data/caltech-101/101_ObjectCategories/lotus/image_0006.jpg"

# Load model (from memory or checkpoint)
if 'ResNet50' in trained_models:
    resnet_model = trained_models['ResNet50']
else:
    resnet_checkpoint = CHECKPOINT_DIR / "ResNet50_best.keras"
    resnet_model = keras.models.load_model(resnet_checkpoint)
    print(f"Loaded ResNet50 from: {resnet_checkpoint}")

# Make prediction
resnet_results = predict_single_image(resnet_model, 'ResNet50', sample_image_path, top_k=5)
display_prediction(resnet_results)

print(f"\nTrue label: {true_label}")

## 16.3 Predict with DenseNet121

In [ ]:
# ============================================
# PREDICT WITH DenseNet121
# ============================================

# Option 1: Use a random test image
# random_idx = np.random.randint(0, len(test_paths))
# sample_image_path = test_paths[random_idx]
# true_label = CLASS_NAMES[test_labels[random_idx]]

# Option 2: Specify your own image path (uncomment and modify)
sample_image_path = "caltech101_data/caltech-101/101_ObjectCategories/lotus/image_0006.jpg"

# Load model (from memory or checkpoint)
if 'DenseNet121' in trained_models:
    densenet_model = trained_models['DenseNet121']
else:
    densenet_checkpoint = CHECKPOINT_DIR / "DenseNet121_best.keras"
    densenet_model = keras.models.load_model(densenet_checkpoint)
    print(f"Loaded DenseNet121 from: {densenet_checkpoint}")

# Make prediction
densenet_results = predict_single_image(densenet_model, 'DenseNet121', sample_image_path, top_k=5)
display_prediction(densenet_results)

print(f"\nTrue label: {true_label}")

## 16.4 Compare All Models on Same Image

In [ ]:
# ============================================
# COMPARE ALL MODELS ON THE SAME IMAGE
# ============================================

# Pick a random test image
random_idx = np.random.randint(0, len(test_paths))
comparison_image_path = test_paths[random_idx]
true_label = CLASS_NAMES[test_labels[random_idx]]

# comparison_image_path = "caltech101_data/caltech-101/101_ObjectCategories/lotus/image_0006.jpg"

print(f"Comparing all models on: {comparison_image_path}")
print(f"True label: {true_label}")

# Show the image
img = plt.imread(comparison_image_path)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis('off')
plt.title(f"True: {true_label}")
plt.show()

# Predict with each model
all_predictions = {}
for model_name in MODELS_TO_TRAIN:
    if model_name in trained_models:
        model = trained_models[model_name]
    else:
        checkpoint = CHECKPOINT_DIR / f"{model_name}_best.keras"
        model = keras.models.load_model(checkpoint)
    
    results = predict_single_image(model, model_name, comparison_image_path, top_k=3)
    all_predictions[model_name] = results['predictions']

# Display comparison
print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(f"\n{'Model':<15} {'Top-1 Prediction':<25} {'Confidence':<12} {'Correct?'}")
print("-" * 70)

for model_name, preds in all_predictions.items():
    top_pred = preds[0]['class']
    confidence = preds[0]['confidence'] * 100
    is_correct = "✓" if top_pred == true_label else "✗"
    print(f"{model_name:<15} {top_pred:<25} {confidence:>8.2f}%    {is_correct}")